In [84]:
# importing inportant libraries
import os
import math
import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib import rc
from sklearn.metrics import confusion_matrix, classification_report
import tkinter as tk
from tkinter import ttk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import zipfile
import requests
import shutil
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import spacy


In [50]:
raw_data1 = pd.read_json(r"C:\UCI\Winter Quarter\Natural Language\Final Project\Data\recipes_raw_nosource_ar.json", orient='records') # read all recipe data
raw_data2 = pd.read_json(r"C:\UCI\Winter Quarter\Natural Language\Final Project\Data\recipes_raw_nosource_epi.json", orient='records')
raw_data3 = pd.read_json(r"C:\UCI\Winter Quarter\Natural Language\Final Project\Data\recipes_raw_nosource_fn.json", orient='records')

raw_data1 = raw_data1.transpose()  # 把行列顛倒
raw_data2 = raw_data2.transpose()
raw_data3 = raw_data3.transpose()

data1 = raw_data1.reset_index(drop=True).iloc[:,:-1]  # data cleaup (去除index亂碼跟picture欄)
data2 = raw_data2.reset_index(drop=True).iloc[:,:-1]
data3 = raw_data3.reset_index(drop=True).iloc[:,:-1]

data_1 = data1.dropna()  # data cleaup (drop data with null values)排除空值資料
data_2 = data2.dropna()  # data cleaup (drop data with null values)排除空值資料
data_3 = data3.dropna()  # data cleaup (drop data with null values)排除空值資料


merged_data = pd.concat([data_1, data_2, data_3], axis=0)
data = merged_data.iloc[:,:-1]
data = data.reset_index(drop=True)  # 重設index (因drop na 會使index有斷層)

data

,title,ingredients,instructions
0,Slow Cooker Chicken and Dumplings,"[4 skinless, boneless chicken breast halves AD...","Place the chicken, butter, soup, and onion in ..."
1,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ..."
2,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar ADVERTISEMENT, 1/2...",Preheat oven to 350 degrees F (175 degrees C)....
3,Best Chocolate Chip Cookies,"[1 cup butter, softened ADVERTISEMENT, 1 cup w...",Preheat oven to 350 degrees F (175 degrees C)....
4,Homemade Mac and Cheese Casserole,[8 ounces whole wheat rotini pasta ADVERTISEME...,Preheat oven to 350 degrees F. Line a 2-quart ...
...,...,...,...
111264,Summer Corn Salad,"[4 ears fresh corn, 2 heads Belgian endive, 2 ...",Watch how to make this recipe.\nPreheat a gril...
111265,Zucchini Stuffed Tomatoes,"[4 large plum tomatoes, Salt and sugar, 1 1/2 ...",Preheat the broiler. Cut the tomatoes in 1/2 c...
111266,Pepper Pasta Quick Cook,"[3 tablespoons olive oil, 2 tablespoons unsalt...",Heat the oil and butter in a large skillet ove...
111267,Chocolate Cake with Armagnac Ice Cream,"[8 ounces butter, 8 ounces bittersweet chocola...",Preheat oven to 350 degrees. On the top half o...


In [51]:
question_data = pd.read_csv(r"C:\UCI\Winter Quarter\Natural Language\Final Project\combined_questions_with_recommended_recipes_1000.csv")
question_data

,Question,Recommended Recipe
0,I have quinoa and avocado and lime. Can you re...,Garlic Chicken Linguine
1,I have pork and apple and cinnamon. Can you re...,Harvest Pork Stew
2,I have salmon and dill and lemon. Can you reco...,Creamy Roasted Vegetable Pasta Salad
3,I have chicken and lemon and garlic. Can you r...,Green Velvet Soup
4,I have lentils and tomato and cumin. Can you r...,Winter Squash Soup with a Sweet Heat
...,...,...
995,I have beef and red wine and rosemary. Can you...,Quick Chicken and Spinach Risotto
996,I have quinoa and avocado and lime. Can you re...,Deconstructed California Roll Bowl
997,I have shrimp and coconut milk and curry powde...,Sweet Italian Chicken Sausage and Tortellini Soup
998,I have eggplant and basil and mozzarella. Can ...,Best Lemon Squares


In [52]:
#data.to_csv('all_recipe.csv', index=False)

In [53]:
#title = data['title'].astype(str) + ' ' + data['ingredients'].astype(str) + ' ' + data['instructions'].astype(str)

#unigram = CountVectorizer(max_df=0.9, min_df=0.0, ngram_range=(1,1))
#bigram = CountVectorizer(max_df=0.9, min_df=0.0, ngram_range=(2,2))
#trigram = CountVectorizer(max_df=0.9, min_df=0.0, ngram_range=(3,3))
#combined = CountVectorizer(max_df=0.9, min_df=0.0, ngram_range=(1,3))
#vectorizers = [unigram, bigram, trigram, combined]
#[vectorizer.fit(title) for vectorizer in vectorizers]

# See vocabulary size
#for vectorizer in vectorizers:
#  print(vectorizer,'Vocabulary Size:', len(vectorizer.vocabulary_))

In [54]:
recipe = data['title'].astype(str) + ' ' + data['ingredients'].astype(str) + ' ' + data['instructions'].astype(str)

tfidf = TfidfVectorizer(ngram_range=(1,3))
tfidf.fit(recipe)


TfidfVectorizer(ngram_range=(1, 3))

In [83]:
# 假設 question_data 是你的 DataFrame，其中包含 'Question' 列
question = [question_data.iloc[50, 0]]  # 提取第 51 行的問題，並將其放入列表中

# 對食譜進行 TF-IDF 轉換
recipe_data = tfidf.fit_transform(recipe)

# 對問題進行 TF-IDF 轉換
X = tfidf.transform(question)

# 計算餘弦相似度
cos_X = cosine_similarity(X, recipe_data)

# 將相似度矩陣降序排序並取前三個最大值的索引
top_indices = np.argsort(cos_X[0])[-3:][::-1]

print(question_data.iloc[50, 0])
print('---------------------------------------------------------------------------')
    
# 打印相似度最高的三筆資料
for i, index in enumerate(top_indices, 1):
    recipe_info = re.split('\[|\]', recipe[index])
    print(f"Recommendation {i} , The similarity is {cos_X[0][index]}:")
    print('*Title:')
    print(recipe_info[0],'\n')
    print('*Ingredients:')
    print(recipe_info[1],'\n')
    print('*Instructions:')
    print(recipe_info[2],'\n')
    print('\n')

I have mushroom and thyme and cream. Can you recommend a sweet dish that can be cooked in short (<30 minutes)?
---------------------------------------------------------------------------
Recommendation 1 , The similarity is 0.289537074272875:
*Title:
Round 2 Recipe - Braised Beef and Mushroom Soup  

*Ingredients:
'1 1/2 cups beef broth', '2 cups drippings, reserved from Holiday Short Ribs', '1/3 cup mushroom and onions, reserved from Mushroom Walnut Tarts', '1 short rib, reserved from Holiday Short Ribs, meat removed and coarsely chopped' 

*Instructions:
 Watch how to make this recipe.
In a medium pot over medium heat, heat the beef broth and drippings. Add the mushrooms and onions. Bring to a simmer, and then stir in the short ribs. Let heat through and serve. 



Recommendation 2 , The similarity is 0.24387569545791069:
*Title:
Five Can Casserole  

*Ingredients:
'1 (6 ounce) can chicken chunks, drained ADVERTISEMENT', '1 (5 ounce) can evaporated milk ADVERTISEMENT', '1 (10.75 ounc

In [56]:
class RecipeRecommendationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Recipe Recommendation System")

        # Style settings for a cleaner look
        style = ttk.Style()
        style.configure('TFrame', background='#E1D8B9')
        style.configure('TButton', background='#F0E4C0')
        style.configure('TLabel', background='#E1D8B9', font=('Arial', 12))
        style.configure('Header.TLabel', font=('Arial', 16, 'bold'))

        # Header
        header_frame = ttk.Frame(root, style='TFrame')
        header_frame.grid(row=0, column=0, columnspan=2, pady=10)
        header_label = ttk.Label(header_frame, text="Recipe Recommendation System", style='Header.TLabel')
        header_label.grid(row=0, column=0, padx=10, pady=10)

        # Input area
        input_frame = ttk.Frame(root, style='TFrame')
        input_frame.grid(row=1, column=0, padx=10, pady=10, sticky='w')
        input_label = ttk.Label(input_frame, text="Enter Your Preferences:", style='TLabel')
        input_label.grid(row=0, column=0, padx=10, pady=5, sticky='w')
        self.input_entry = ttk.Entry(input_frame, width=40)
        self.input_entry.grid(row=1, column=0, padx=10, pady=5)

        # Run button
        run_button = ttk.Button(root, text="Get Recipe Recommendation", command=self.get_recommendation, style='TButton')
        run_button.grid(row=1, column=1, padx=10, pady=10, sticky='e')

        # Output area
        output_frame = ttk.Frame(root, style='TFrame')
        output_frame.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky='w')
        output_label = ttk.Label(output_frame, text="Recipe Recommendation:", style='TLabel')
        output_label.grid(row=0, column=0, padx=10, pady=5, sticky='w')
        self.result_var = tk.StringVar()
        self.output_entry = ttk.Entry(output_frame, textvariable=self.result_var, state='readonly', width=40)
        self.output_entry.grid(row=1, column=0, padx=10, pady=5)

    def get_recommendation(self):
        # Placeholder for calling your recommendation system based on user input
        user_input = self.input_entry.get()
        # Replace this with the actual logic of your recommendation system
        recommendation = f"Recommended recipe for: {user_input}"

        # Update the output label
        self.result_var.set(recommendation)

#if __name__ == "__main__":  # 等最後再把#拿掉
    root = tk.Tk()
    app = RecipeRecommendationApp(root)
    root.mainloop()


NameError: name 'RecipeRecommendationApp' is not defined